In [ ]:
# Descomenta y ejecuta esta celda si no tienes las bibliotecas instaladas
# 
%pip install vedo numpy pandas trimesh open3d


In [ ]:

# Importar bibliotecas necesarias
import vedo       # Para visualización y creación de objetos 3D
import numpy as np # Para manejar arrays de coordenadas numéricas
import random     # Para generar variaciones aleatorias (ej. color, tamaño)
import os         # Para manejar archivos (necesario para el bonus y exportación)

# Opcional: Importar otras bibliotecas si se quieren usar para exportar o el bonus
import trimesh
import open3d as o3d
import pandas as pd # Para el bonus con CSV

print("Bibliotecas importadas correctamente.")

In [ ]:
# 1. Definir la lista de coordenadas 3D
# Usamos un array de NumPy para facilitar operaciones matemáticas si fueran necesarias
coordenadas = np.array([
    [0, 0, 0],   
    [5, 0, 0],    
    [0, 5, 0],    
    [0, 0, 5],    
    [5, 5, 0],    
    [5, 0, 5],    
    [0, 5, 5],   
    [5, 5, 5],   
    [2.5, 2.5, -2] 
])

print("Coordenadas 3D definidas:")
print(coordenadas)

In [ ]:
# 2. Inicializar la escena de vedo y una lista para guardar los objetos
escena = vedo.Plotter(title="Taller Escena Paramétrica", axes=1) # axes=1 muestra los ejes XYZ
# axes=1 dibuja un cubo con ejes etiquetados
# axes=4 dibuja los ejes X, Y, Z desde el origen

objetos_generados = [] # Lista para almacenar los objetos vedo creados

print("Escena vedo inicializada.")

In [ ]:
# 3. Iterar sobre las coordenadas y generar objetos
print("\n--- Generando objetos ---")

for i, punto in enumerate(coordenadas):
    print(f"\nProcesando punto {i}: {punto}")

    # --- Lógica condicional para variar parámetros ---
    forma = "cubo"      # Forma por defecto
    tamano = 0.5        # Tamaño base
    # Color aleatorio para cada objeto
    color = random.choice(["red", "green", "blue", "yellow", "purple", "orange", "cyan", "magenta"])

    # Ejemplo 1: Cambiar la FORMA basada en el índice (par/impar)
    if i % 2 == 0:
        forma = "esfera"
        print(f"  Índice par ({i}): Se creará una ESFERA.")
    else:
        print(f"  Índice impar ({i}): Se creará un CUBO.")

    # Ejemplo 2: Cambiar el TAMAÑO basado en la coordenada Z
    if punto[2] > 3:
        tamano = 1.0 # Más grande si está alto en Z
        print(f"  Coordenada Z ({punto[2]}) > 3: Tamaño aumentado a {tamano}.")
    elif punto[2] < 0:
        tamano = 0.2 # Más pequeño si está por debajo del plano XY
        forma = "cilindro" # Cambiamos también la forma para Z < 0
        print(f"  Coordenada Z ({punto[2]}) < 0: Tamaño reducido a {tamano} y forma cambiada a CILINDRO.")
    else:
        print(f"  Coordenada Z ({punto[2]}) en rango normal: Tamaño {tamano}.")

    # --- Crear la primitiva 3D seleccionada ---
    objeto_3d = None # Inicializar la variable del objeto

    if forma == "cubo":
        # vedo.Cube() crea un cubo centrado en (0,0,0) con lado 1.
        # Lo posicionamos (.pos()) y escalamos (.scale()) o definimos el lado (side=)
        # Usaremos side= para simplicidad aquí.
        objeto_3d = vedo.Cube(pos=punto, side=tamano, c=color)
        print(f"  Creado Cubo en {punto}, tamaño={tamano}, color={color}")

    elif forma == "esfera":
        # vedo.Sphere() crea una esfera centrada en (0,0,0) con radio 0.5.
        # Lo posicionamos (.pos()) y definimos el radio (r=)
        objeto_3d = vedo.Sphere(pos=punto, r=tamano, c=color)
        print(f"  Creada Esfera en {punto}, radio={tamano}, color={color}")

    elif forma == "cilindro":
        # vedo.Cylinder() crea un cilindro.
        # r=radio, h=altura. Lo posicionamos (.pos())
        # Haremos la altura el doble del tamaño base para que se vea bien.
        objeto_3d = vedo.Cylinder(pos=punto, r=tamano, height=tamano*2, c=color)
        print(f"  Creado Cilindro en {punto}, radio={tamano}, altura={tamano*2}, color={color}")

    # --- Añadir el objeto creado a nuestra lista y a la escena ---
    if objeto_3d: # Solo si se creó un objeto (podríamos tener condiciones donde no se cree)
        objetos_generados.append(objeto_3d) # Guardar referencia en la lista
        # escena.add(objeto_3d) # Añadir el objeto a la escena para visualización

print("\n--- Fin de la generación de objetos ---")
print(f"Total de objetos generados: {len(objetos_generados)}")

# Añadir todos los objetos generados a la escena de una vez
escena.add(objetos_generados)

In [ ]:
# 4. Mostrar la escena interactiva
print("\nMostrando la escena generada...")

# Muestra la ventana interactiva.
# El script se pausará aquí hasta que cierres manualmente la ventana 3D.
escena.show(interactive=True)



In [ ]:
# Crear directorio de exportación si no existe
output_dir = "exports"
os.makedirs(output_dir, exist_ok=True)

# Combinar todos los objetos 3D en una sola malla real
malla_combinada = vedo.merge(*objetos_generados)

# Exportar como OBJ y STL usando vedo
vedo.write(malla_combinada, os.path.join(output_dir, "escena_vedo.obj"))
vedo.write(malla_combinada, os.path.join(output_dir, "escena_vedo.stl"))

print("Escena exportada con vedo.write() en formatos OBJ y STL.")


In [ ]:
# Convertir con Trimesh (y usar 'mesh.cells' en lugar de 'mesh.faces()')
trimesh_objs = []
for vobj in objetos_generados:
    # Convertir cada objeto vedo en malla trimesh
    mesh_tri = vedo.Mesh(vobj).triangulate()  # Triangular las caras
    vertices = mesh_tri.points
    faces = mesh_tri.cells[0]  # Usar cells en lugar de faces() para obtener las caras triangulares
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
    trimesh_objs.append(mesh)

# Crear una escena de Trimesh y exportar
scene = trimesh.Scene(trimesh_objs)
scene.export(os.path.join(output_dir, "escena_trimesh.stl"))


In [ ]:
#Exportar usando Open3D
for vobj in objetos_generados:
    # Triangular el objeto vedo
    mesh_tri = vedo.Mesh(vobj).triangulate()
    
    # Convertir a formato NumPy
    vertices = np.asarray(mesh_tri.points, dtype=np.float64)
    faces = np.asarray(mesh_tri.cells, dtype=np.int32)  # Cambié la llamada de cells() a cells
    
    # Crear malla Open3D
    mesh_o3d = o3d.geometry.TriangleMesh()
    mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_o3d.triangles = o3d.utility.Vector3iVector(faces)
    mesh_o3d.compute_vertex_normals()
    
    # Exportar como PLY
    o3d.io.write_triangle_mesh(os.path.join(output_dir, "escena_open3d.ply"), mesh_o3d)


In [ ]:
# Leer datos desde .csv o .json para automatizar la generación y exportación.

import csv
import json
import vedo
import trimesh
import open3d as o3d
import numpy as np
import random
import os

# Leer datos desde un archivo CSV
def leer_csv(ruta_csv):
    datos = []
    with open(ruta_csv, mode='r') as archivo:
        lector = csv.DictReader(archivo)
        for fila in lector:
            datos.append({
                "x": float(fila["x"]),
                "y": float(fila["y"]),
                "z": float(fila["z"]),
                "forma": fila["forma"],
                "color": fila["color"],
                "tamano": float(fila["tamano"])
            })
    return datos

# Leer datos desde un archivo JSON
def leer_json(ruta_json):
    with open(ruta_json, "r") as archivo:
        return json.load(archivo)

# Selección de archivo (CSV o JSON)
archivo_entrada = "../datos/datos_escena.csv"  # Cambia a "data.json" para usar el archivo JSON

if archivo_entrada.endswith('.csv'):
    datos = leer_csv(archivo_entrada)
elif archivo_entrada.endswith('.json'):
    datos = leer_json(archivo_entrada)

# Crear la escena vedo
escena = vedo.Plotter(title="Escena 3D Generada Automáticamente", axes=1)
objetos_generados = []

# Procesar los datos y generar los objetos 3D
for i, dato in enumerate(datos):
    # Coordenadas 3D
    punto = [dato["x"], dato["y"], dato["z"]]
    
    # Forma y parámetros
    forma = dato["forma"]
    color = dato["color"]
    tamano = dato["tamano"]

    print(f"\nProcesando punto {i}: {punto} - Forma: {forma}, Color: {color}, Tamaño: {tamano}")

    # Crear el objeto correspondiente según la forma
    if forma == "cubo":
        objeto_3d = vedo.Cube(pos=punto, side=tamano, c=color)
    elif forma == "esfera":
        objeto_3d = vedo.Sphere(pos=punto, r=tamano, c=color)
    elif forma == "cilindro":
        objeto_3d = vedo.Cylinder(pos=punto, r=tamano, height=tamano*2, c=color)
    
    # Añadir el objeto generado a la lista
    objetos_generados.append(objeto_3d)

# Combinar todos los objetos en una sola malla para exportar
malla_combinada = vedo.merge(*objetos_generados)

# Crear directorio de exportación si no existe
output_dir = "exports"
os.makedirs(output_dir, exist_ok=True)

# Exportar la malla combinada usando vedo (como OBJ y STL)
vedo.write(malla_combinada, os.path.join(output_dir, "escena_automatica.obj"))
vedo.write(malla_combinada, os.path.join(output_dir, "escena_automatica.stl"))

# Exportación usando trimesh
trimesh_objs = []
for objeto in objetos_generados:
    mesh_tri = vedo.Mesh(objeto).triangulate()  # Triangular las caras
    vertices = mesh_tri.points
    faces = mesh_tri.cells  # Usar cells en lugar de faces
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
    trimesh_objs.append(mesh)

# Crear una escena de Trimesh y exportar
scene = trimesh.Scene(trimesh_objs)
scene.export(os.path.join(output_dir, "scene_trimesh.stl"))

# Exportación usando Open3D
for objeto in objetos_generados:
    mesh_tri = vedo.Mesh(objeto).triangulate()
    
    vertices = np.asarray(mesh_tri.points, dtype=np.float64)
    faces = np.asarray(mesh_tri.cells, dtype=np.int32)
    
    mesh_o3d = o3d.geometry.TriangleMesh()
    mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_o3d.triangles = o3d.utility.Vector3iVector(faces)
    mesh_o3d.compute_vertex_normals()
    
    o3d.io.write_triangle_mesh(os.path.join(output_dir, "mesh_open3d.ply"), mesh_o3d)

print("\nExportación completada.")
